# Test the effect of damping

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib
from numpy import random
import copy
import scipy

In [2]:
matplotlib.rc("font", **{"family":"serif","serif":["Computer Modern Roman"]})
matplotlib.rc("text", usetex=True)

## Toy model data

In [3]:
# simulation parameters
# toy model

numx = 30
numy = 30
N = numx*numy
R = 0.5

spacing = 1.0
dx = 0.5*spacing
dy = 0.5*spacing
Lx = spacing * 2 * R * numx
Ly = 0.5 * np.sqrt(3) * numy * spacing * 2 * R
xnum = round(Lx/dx)
ynum = round(Ly/dy)

nrec = 1000
n_j = int(nrec*0.8)
wnum = int(n_j/2)+1

dkx = 2*np.pi/Lx
dky = 2*np.pi/Ly

dt = 0.1
dw = 2*np.pi/(dt*n_j)

In [4]:
# load current correlation files
# toy model damped

with open('Cll_toy_damped_k05_ka1_D0.dat','rb') as f:
    Cll = np.fromfile(f,np.float64)
Cll = (1/N)*Cll.reshape((xnum,ynum,wnum))

with open('Cltr_toy_damped_k05_ka1_D0.dat','rb') as f:
    Cltr = np.fromfile(f,np.float64)
Cltr = (1/N)*Cltr.reshape((xnum,ynum,wnum))

with open('Clti_toy_damped_k05_ka1_D0.dat','rb') as f:
    Clti = np.fromfile(f,np.float64)
Clti = (1/N)*Clti.reshape((xnum,ynum,wnum))

with open('Ctt_toy_damped_k05_ka1_D0.dat','rb') as f:
    Ctt = np.fromfile(f,np.float64)
Ctt = (1/N)*Ctt.reshape((xnum,ynum,wnum))


In [20]:
# load current correlation files
# toy model undamped

with open('Cll_toy_undamped_k0_ka1_D0.dat','rb') as f:
    Cll = np.fromfile(f,np.float64)
Cll = (1/N)*Cll.reshape((xnum,ynum,wnum))

with open('Cltr_toy_undamped_k0_ka1_D0.dat','rb') as f:
    Cltr = np.fromfile(f,np.float64)
Cltr = (1/N)*Cltr.reshape((xnum,ynum,wnum))

with open('Clti_toy_undamped_k0_ka1_D0.dat','rb') as f:
    Clti = np.fromfile(f,np.float64)
Clti = (1/N)*Clti.reshape((xnum,ynum,wnum))

with open('Ctt_toy_undamped_k0_ka1_D0.dat','rb') as f:
    Ctt = np.fromfile(f,np.float64)
Ctt = (1/N)*Ctt.reshape((xnum,ynum,wnum))

## Starfish model data

In [3]:
# simulation parameters
# starfish model

numx = 30
numy = 30
N = numx*numy
R = 0.5

spacing = 1.2
dx = 0.5*spacing
dy = 0.5*spacing
Lx = spacing * 2 * R * numx
Ly = 0.5 * np.sqrt(3) * numy * spacing * 2 * R
xnum = round(Lx/dx)
ynum = round(Ly/dy)

nrec = 1000
n_j = int(nrec*0.8)
wnum = int(n_j/2)+1

dkx = 2*np.pi/Lx
dky = 2*np.pi/Ly

dt = 0.1
dw = 2*np.pi/(dt*n_j)

In [11]:
# load current correlation files
# starfish model undamped

with open('Cll_starfish_undamped.dat','rb') as f:
    Cll = np.fromfile(f,np.float64)
Cll = (1/N)*Cll.reshape((xnum,ynum,wnum))

with open('Cltr_starfish_undamped.dat','rb') as f:
    Cltr = np.fromfile(f,np.float64)
Cltr = (1/N)*Cltr.reshape((xnum,ynum,wnum))

with open('Clti_starfish_undamped.dat','rb') as f:
    Clti = np.fromfile(f,np.float64)
Clti = (1/N)*Clti.reshape((xnum,ynum,wnum))

with open('Ctt_starfish_undamped.dat','rb') as f:
    Ctt = np.fromfile(f,np.float64)
Ctt = (1/N)*Ctt.reshape((xnum,ynum,wnum))

In [4]:
# load current correlation files
# starfish model damped

with open('Cll_starfish_damped.dat','rb') as f:
    Cll = np.fromfile(f,np.float64)
Cll = (1/N)*Cll.reshape((xnum,ynum,wnum))

with open('Cltr_starfish_damped.dat','rb') as f:
    Cltr = np.fromfile(f,np.float64)
Cltr = (1/N)*Cltr.reshape((xnum,ynum,wnum))

with open('Clti_starfish_damped.dat','rb') as f:
    Clti = np.fromfile(f,np.float64)
Clti = (1/N)*Clti.reshape((xnum,ynum,wnum))

with open('Ctt_starfish_damped.dat','rb') as f:
    Ctt = np.fromfile(f,np.float64)
Ctt = (1/N)*Ctt.reshape((xnum,ynum,wnum))

### frequency domain

In [5]:
# frequency domain 

kxrange = np.zeros(xnum)
kyrange = np.zeros(ynum)
wrange = dw*np.arange(wnum)

for i in range(xnum):
    kxrange[i] = -np.pi/dx + dkx*i
for j in range(ynum):
    kyrange[j] = -np.pi/dy + dky*j
    
kx,ky = np.meshgrid(kxrange,kyrange,indexing='ij')

## Plot of each C tensor element

In [6]:
# 2d matrix whose elements are the max value of C at particular (qx,qy)
# for auto-correlation

Cll_copy = copy.copy(Cll)
Cltr_copy = copy.copy(Cltr)
Clti_copy = copy.copy(Clti)
Ctt_copy = copy.copy(Ctt)

Cll_peak = np.zeros([xnum,ynum])
Cltr_peak = np.zeros([xnum,ynum])
Clti_peak = np.zeros([xnum,ynum])
Ctt_peak = np.zeros([xnum,ynum])

wll_peak = np.zeros([xnum,ynum])
wltr_peak = np.zeros([xnum,ynum])
wlti_peak = np.zeros([xnum,ynum])
wtt_peak = np.zeros([xnum,ynum])

fromhere = 1

for i in range(xnum):
    for j in range(ynum):
        Cll_thisq = Cll_copy[i,j,fromhere:]
        Ctt_thisq = Ctt_copy[i,j,fromhere:]
        Cltr_thisq = Cltr_copy[i,j,fromhere:]
        Clti_thisq = Clti_copy[i,j,fromhere:]
        Cltr_thisq_pos = abs(Cltr_copy[i,j,fromhere:])
        Clti_thisq_pos = abs(Clti_copy[i,j,fromhere:])
        
        Cll_peak[i,j] = np.max(Cll_thisq) # record max C value at certain q
        idx_wllmax = np.where(Cll_thisq==np.max(Cll_thisq)) # index in w dimension for the max C
        idx_wllmax = int(idx_wllmax[0][0]) + fromhere # make the index an integer and add fromhere
        wll_peak[i,j] = wrange[idx_wllmax] # record the value of max peak w. 
        
        Ctt_peak[i,j] = np.max(Ctt_thisq) # record max C value at certain q
        idx_wttmax = np.where(Ctt_thisq==np.max(Ctt_thisq)) # index in w dimension for the max C
        idx_wttmax = int(idx_wttmax[0][0]) + fromhere # make the index an integer and add fromhere
        wtt_peak[i,j] = wrange[idx_wttmax] # record the value of max peak w. 
        
        idx_wltrmax = np.where(Cltr_thisq_pos==np.max(Cltr_thisq_pos))
        idx_wltrmax = int(idx_wltrmax[0][0]) + fromhere
        wltr_peak[i,j] = wrange[idx_wltrmax]
        Cltr_peak[i,j] = Cltr_thisq[idx_wltrmax - fromhere]
        
        idx_wltimax = np.where(Clti_thisq_pos==np.max(Clti_thisq_pos))
        idx_wltimax = int(idx_wltimax[0][0]) + fromhere
        wlti_peak[i,j] = wrange[idx_wltimax]
        Clti_peak[i,j] = Clti_thisq[idx_wltimax - fromhere]
        

In [ ]:
Cll_plot = copy.copy(Cll_peak)
wll_plot = copy.copy(wll_peak)
Ctt_plot = copy.copy(Ctt_peak)
wtt_plot = copy.copy(wtt_peak)
Cltr_plot = copy.copy(Cltr_peak)
wltr_plot = copy.copy(wltr_peak)
Clti_plot = copy.copy(Clti_peak)
wlti_plot = copy.copy(wlti_peak)

peak_thres = 0

kx_ll = kx[np.abs(Cll_plot)>peak_thres]
ky_ll = ky[np.abs(Cll_plot)>peak_thres]
wpk_ll = wll_plot[np.abs(Cll_plot)>peak_thres]

kx_tt = kx[np.abs(Ctt_plot)>peak_thres]
ky_tt = ky[np.abs(Ctt_plot)>peak_thres]
wpk_tt = wtt_plot[np.abs(Ctt_plot)>peak_thres]

kx_ltr = kx[np.abs(Cltr_plot)>peak_thres]
ky_ltr = ky[np.abs(Cltr_plot)>peak_thres]
wpk_ltr = wltr_plot[np.abs(Cltr_plot)>peak_thres]

kx_lti = kx[np.abs(Clti_plot)>peak_thres]
ky_lti = ky[np.abs(Clti_plot)>peak_thres]
wpk_lti = wlti_plot[np.abs(Clti_plot)>peak_thres]

%matplotlib inline

fig = plt.figure(facecolor=(1,1,1),figsize=(23,15))
ax1 = fig.add_subplot(2,2,1,projection='3d')
ax2 = fig.add_subplot(2,2,2,projection='3d')
ax3 = fig.add_subplot(2,2,3,projection='3d')
ax4 = fig.add_subplot(2,2,4,projection='3d')
fig.tight_layout()

# first plot Cll
img1 = ax1.scatter3D(kx,ky,wll_plot,c=Cll_plot,cmap = 'viridis',s=10,rasterized=True)

cb1 = ax1.figure.colorbar(img1,shrink=0.7,pad=0.08)
ax1.set_xlabel('$q_x$', fontsize=20, labelpad=10)
ax1.set_ylabel('$q_y$', fontsize=20, labelpad=10)
ax1.set_zlabel('$\omega$', fontsize=20, labelpad=10)
cb1.set_label('$C$',fontsize=20)

ax1.text2D(-0.08,0.06,'(a)',fontsize=25)
ax1.tick_params(labelsize=15)
cb1.ax.tick_params(labelsize=15)

# second plot Ctt
img2 = ax2.scatter3D(kx,ky,wtt_plot,c=Ctt_plot,cmap = 'viridis',s=10,rasterized=True)

cb2 = ax2.figure.colorbar(img2,shrink=0.7,pad=0.08)
ax2.set_xlabel('$q_x$', fontsize=20, labelpad=10)
ax2.set_ylabel('$q_y$', fontsize=20, labelpad=10)
ax2.set_zlabel('$\omega$', fontsize=20, labelpad=10)
cb2.set_label('$C$',fontsize=20)

ax2.text2D(-0.08,0.06,'(b)',fontsize=25)
ax2.tick_params(labelsize=15)
cb2.ax.tick_params(labelsize=15)

# third plot Cltr
img3 = ax3.scatter3D(kx,ky,wltr_plot,c=Cltr_plot,cmap = 'viridis',s=10,rasterized=True)

cb3 = ax3.figure.colorbar(img3,shrink=0.7,pad=0.08)
ax3.set_xlabel('$q_x$', fontsize=20, labelpad=10)
ax3.set_ylabel('$q_y$', fontsize=20, labelpad=10)
ax3.set_zlabel('$\omega$', fontsize=20, labelpad=10)
cb3.set_label('$C$',fontsize=20)

ax3.text2D(-0.08,0.06,'(c)',fontsize=25)
ax3.tick_params(labelsize=15)
cb3.ax.tick_params(labelsize=15)

# fourth plot Clti
img4 = ax4.scatter3D(kx,ky,wlti_plot,c=Clti_plot,cmap = 'viridis',s=10,rasterized=True)

cb4 = ax4.figure.colorbar(img4,shrink=0.7,pad=0.08)
ax4.set_xlabel('$q_x$', fontsize=20, labelpad=10)
ax4.set_ylabel('$q_y$', fontsize=20, labelpad=10)
ax4.set_zlabel('$\omega$', fontsize=20, labelpad=10)
cb4.set_label('$C$',fontsize=20)

ax4.text2D(-0.08,0.06,'(d)',fontsize=25)
ax4.tick_params(labelsize=15)
cb4.ax.tick_params(labelsize=15)

#plt.savefig('noNoise_Damp_starfish_v1.pdf',dpi=600,bbox_inches='tight')
#plt.savefig('noNoise_Damp_toy_test.pdf',dpi=200,bbox_inches='tight')
plt.show()
